In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 64kB/s 
     |████████████████████████████████| 204kB 18.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=22fe2fe781c9bb4d79dc3a0529320f43dbdc59f21a71c92dbb695b96a48a7598
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [135]:
# importing necessary libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window


In [136]:
# initialise spark session

spark = SparkSession.builder.appName('Jobathon').getOrCreate()

In [137]:
# read VisitorLog data

visitorlog_df = spark.read.csv('VisitorLogsData.csv',header=True,inferSchema=True)
visitorlog_df.show()

+--------------+--------------------+---------+-------+--------+-------------+--------+-----------+--------------------+
|   webClientID|       VisitDateTime|ProductID| UserID|Activity|      Browser|      OS|       City|             Country|
+--------------+--------------------+---------+-------+--------+-------------+--------+-----------+--------------------+
| WI10000050298|2018-05-07 04:28:...| pr100631|   null|    null|Chrome Mobile| Android|    Chennai|               India|
| WI10000025922|2018-05-13 07:26:...| pr100707|   null|    null|       Chrome| Windows|       null|              Taiwan|
|WI100000204522|2018-05-11 11:43:...| pr100030|   null|   click|       Chrome| windows|    Gurgaon|               India|
| WI10000011974|2018-05-13 15:20:...| Pr100192|   null|   CLICK|       Chrome| Windows|           |                    |
|WI100000441953|2018-05-08 20:44:...| Pr100762|   null|   click|       Chrome|mac os x|     Iselin|       United States|
| WI10000042224|2018-05-24 13:10

In [138]:
# read User data

user_df = spark.read.csv('userTable.csv',header=True,inferSchema=True)
user_df.show()

+-------+--------------------+------------+
| UserID|         Signup Date|User Segment|
+-------+--------------------+------------+
|U133159|2018-04-14 07:01:...|           C|
|U129368|2017-12-02 09:38:...|           B|
|U109654|2013-03-19 11:38:...|           B|
|U108998|2018-01-18 08:29:...|           C|
|U131393|2018-03-27 08:05:...|           B|
|U101889|2018-01-24 12:50:...|           B|
|U113233|2017-03-18 09:27:...|           B|
|U115728|2017-08-18 11:40:...|           B|
|U132899|2018-03-14 08:32:...|           B|
|U107492|2018-04-10 20:42:...|           B|
|U124573|2014-09-25 07:24:...|           B|
|U121993|2018-03-30 05:49:...|           B|
|U125731|2017-12-22 16:57:...|           B|
|U102932|2018-02-10 20:30:...|           C|
|U114104|2018-04-01 03:31:...|           C|
|U115212|2016-03-10 04:56:...|           B|
|U127832|2018-03-22 15:30:...|           B|
|U129967|2018-02-17 04:20:...|           B|
|U102610|2017-06-05 05:14:...|           A|
|U112354|2017-09-02 12:47:...|  

In [139]:
# VisitorLog data info

visitorlog_df.printSchema()

root
 |-- webClientID: string (nullable = true)
 |-- VisitDateTime: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- UserID: string (nullable = true)
 |-- Activity: string (nullable = true)
 |-- Browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)



In [140]:
# User data info

user_df.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- Signup Date: string (nullable = true)
 |-- User Segment: string (nullable = true)



In [141]:
# statistical description

visitorlog_df.describe().show()

+-------+--------------+--------------------+---------+-------+--------+-------------+-------+-------+---------------+
|summary|   webClientID|       VisitDateTime|ProductID| UserID|Activity|      Browser|     OS|   City|        Country|
+-------+--------------+--------------------+---------+-------+--------+-------------+-------+-------+---------------+
|  count|       6588000|             5929085|  6060863| 650695| 5698554|      6588000|6588000|4422169|        6190307|
|   mean|          null|1.526554171346754...|     null|   null|    null|         null|   null|   null|           null|
| stddev|          null|5.338116776808671E14|     null|   null|    null|         null|   null|   null|           null|
|    min|     WI1000000| 1525651211454000000| Pr100000|U100002|   CLICK|AdsBot-Google|Android|       |               |
|    max|WI100000999999|2018-05-27 23:59:...| pr112449|U136965|pageload|      net/bot|windows|  ‘Aiea|Åland Mariehamn|
+-------+--------------+--------------------+---

In [142]:
# statistical description

user_df.describe().show()

+-------+-------+--------------------+------------+
|summary| UserID|         Signup Date|User Segment|
+-------+-------+--------------------+------------+
|  count|  34050|               34050|       34050|
|   mean|   null|                null|        null|
| stddev|   null|                null|        null|
|    min|U100002|2010-12-01 03:11:...|           A|
|    max|U136965|2018-05-20 07:06:...|           C|
+-------+-------+--------------------+------------+



In [143]:
# combining VisitorLog and User tables

join_df = visitorlog_df.join(user_df,['UserID'],'inner')
join_df.show()

+-------+--------------+--------------------+---------+--------+-------------+---------+----------------+--------------------+--------------------+------------+
| UserID|   webClientID|       VisitDateTime|ProductID|Activity|      Browser|       OS|            City|             Country|         Signup Date|User Segment|
+-------+--------------+--------------------+---------+--------+-------------+---------+----------------+--------------------+--------------------+------------+
|U106593|WI100000112772| 1526366895249000000| Pr100017|   click|Chrome Mobile|  Android|                |                    |2017-09-10 15:48:...|           B|
|U108297|  WI1000009977| 1527058921790000000| Pr101008|    null|Chrome Mobile|  Android|           Delhi|               India|2015-05-26 04:52:...|           B|
|U132443|WI100000223826|2018-05-10 06:28:...| Pr100241|    null|      Firefox|  Windows|            null|               India|2017-09-29 18:26:...|           C|
|U134616| WI10000021998|2018-05-08

In [144]:
join_df.describe().show()

+-------+-------+--------------+--------------------+---------+--------+--------------+-------+------+---------------+--------------------+------------+
|summary| UserID|   webClientID|       VisitDateTime|ProductID|Activity|       Browser|     OS|  City|        Country|         Signup Date|User Segment|
+-------+-------+--------------+--------------------+---------+--------+--------------+-------+------+---------------+--------------------+------------+
|  count| 650695|        650695|              585892|   598732|  562938|        650695| 650695|414571|         607868|              650695|      650695|
|   mean|   null|          null|1.526549450754298...|     null|    null|          null|   null|  null|           null|                null|        null|
| stddev|   null|          null| 5.46457639900986E14|     null|    null|          null|   null|  null|           null|                null|        null|
|    min|U100002|     WI1000000| 1525651214036000000| Pr100000|   CLICK|   Amazon 

In [145]:
# dropping irrelevant columns
 
join_df = join_df.drop('Country','City','User Segment')
join_df.show()

+-------+--------------+--------------------+---------+--------+-------------+---------+--------------------+
| UserID|   webClientID|       VisitDateTime|ProductID|Activity|      Browser|       OS|         Signup Date|
+-------+--------------+--------------------+---------+--------+-------------+---------+--------------------+
|U106593|WI100000112772| 1526366895249000000| Pr100017|   click|Chrome Mobile|  Android|2017-09-10 15:48:...|
|U108297|  WI1000009977| 1527058921790000000| Pr101008|    null|Chrome Mobile|  Android|2015-05-26 04:52:...|
|U132443|WI100000223826|2018-05-10 06:28:...| Pr100241|    null|      Firefox|  Windows|2017-09-29 18:26:...|
|U134616| WI10000021998|2018-05-08 12:40:...| pr100495|   click|       Chrome|  Windows|2017-03-06 15:04:...|
|U130784| WI10000036281|2018-05-11 15:35:...| Pr100363|   click|       Chrome|Chrome OS|2017-09-07 18:44:...|
|U120983|WI100000110130|2018-05-19 00:02:...| pr100340|   click|       Chrome|  Windows|2018-03-11 05:06:...|
|U120287| 

In [146]:
# checking count of unique UserID's

join_df.select('UserID').distinct().count()

34050

In [147]:
# checking shape of data
 
print((join_df.count(), len(join_df.columns)))

(650695, 8)


In [148]:
# checking count of null values in each column

join_df.select([count(when(isnull(c), c)).alias(c) for c in join_df.columns]).show()

+------+-----------+-------------+---------+--------+-------+---+-----------+
|UserID|webClientID|VisitDateTime|ProductID|Activity|Browser| OS|Signup Date|
+------+-----------+-------------+---------+--------+-------+---+-----------+
|     0|          0|        64803|    51963|   87757|      0|  0|          0|
+------+-----------+-------------+---------+--------+-------+---+-----------+



In [149]:
# converting VisitDatetime & Signup Date timestamps to date format

join_df = join_df.withColumn('VisitDateTime', when(length('VisitDateTime')==19, to_date(from_unixtime(col('VisitDateTime')/(10**9),'yyyy-MM-dd'))).otherwise(to_date(col('VisitDateTime'))))
join_df = join_df.withColumn('Signup Date',to_date(col('Signup Date')))

join_df.show()

+-------+--------------+-------------+---------+--------+-------------+---------+-----------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|      Browser|       OS|Signup Date|
+-------+--------------+-------------+---------+--------+-------------+---------+-----------+
|U106593|WI100000112772|   2018-05-15| Pr100017|   click|Chrome Mobile|  Android| 2017-09-10|
|U108297|  WI1000009977|   2018-05-23| Pr101008|    null|Chrome Mobile|  Android| 2015-05-26|
|U132443|WI100000223826|   2018-05-10| Pr100241|    null|      Firefox|  Windows| 2017-09-29|
|U134616| WI10000021998|   2018-05-08| pr100495|   click|       Chrome|  Windows| 2017-03-06|
|U130784| WI10000036281|   2018-05-11| Pr100363|   click|       Chrome|Chrome OS| 2017-09-07|
|U120983|WI100000110130|   2018-05-19| pr100340|   click|       Chrome|  Windows| 2018-03-11|
|U120287| WI10000014616|   2018-05-19| Pr100166|   click|       Chrome|  Windows| 2018-04-11|
|U124307|WI100000102012|   2018-05-07| pr101042|   CLICK|   

In [150]:
join_df.select('Activity').distinct().collect()

[Row(Activity=None),
 Row(Activity='CLICK'),
 Row(Activity='click'),
 Row(Activity='PAGELOAD'),
 Row(Activity='pageload')]

In [151]:
# converting Activity column values to uppercase

join_df = join_df.withColumn('Activity', upper(col('Activity')))
join_df.show()

+-------+--------------+-------------+---------+--------+-------------+---------+-----------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|      Browser|       OS|Signup Date|
+-------+--------------+-------------+---------+--------+-------------+---------+-----------+
|U106593|WI100000112772|   2018-05-15| Pr100017|   CLICK|Chrome Mobile|  Android| 2017-09-10|
|U108297|  WI1000009977|   2018-05-23| Pr101008|    null|Chrome Mobile|  Android| 2015-05-26|
|U132443|WI100000223826|   2018-05-10| Pr100241|    null|      Firefox|  Windows| 2017-09-29|
|U134616| WI10000021998|   2018-05-08| pr100495|   CLICK|       Chrome|  Windows| 2017-03-06|
|U130784| WI10000036281|   2018-05-11| Pr100363|   CLICK|       Chrome|Chrome OS| 2017-09-07|
|U120983|WI100000110130|   2018-05-19| pr100340|   CLICK|       Chrome|  Windows| 2018-03-11|
|U120287| WI10000014616|   2018-05-19| Pr100166|   CLICK|       Chrome|  Windows| 2018-04-11|
|U124307|WI100000102012|   2018-05-07| pr101042|   CLICK|   

In [152]:
# Activity column null value imputation

activity_counts = join_df.groupBy(['UserID', 'Activity']).agg(count(when(col('Activity').isNotNull(), col('Activity') ).otherwise(None)).alias('count'))

win = Window().partitionBy('UserID').orderBy(col('count').desc())
result = (activity_counts
          .withColumn('row_num', row_number().over(win))
          .where(col('row_num') == 1)
          .select('UserID', 'Activity')
         )
result = result.withColumnRenamed('Activity','Activity_New')
result.show()

+-------+------------+
| UserID|Activity_New|
+-------+------------+
|U100364|       CLICK|
|U100499|       CLICK|
|U100608|    PAGELOAD|
|U100611|    PAGELOAD|
|U100754|       CLICK|
|U100999|       CLICK|
|U102315|       CLICK|
|U103221|    PAGELOAD|
|U103279|    PAGELOAD|
|U103647|    PAGELOAD|
|U103859|       CLICK|
|U103874|    PAGELOAD|
|U103924|    PAGELOAD|
|U104238|       CLICK|
|U104440|       CLICK|
|U104625|    PAGELOAD|
|U104644|       CLICK|
|U104722|    PAGELOAD|
|U105187|    PAGELOAD|
|U105380|    PAGELOAD|
+-------+------------+
only showing top 20 rows



In [153]:
join_df = join_df.join(result,['UserID'],'inner')
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+------------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|Activity_New|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+------------+
|U100364|WI100000401415|   2018-05-14| pr100152|   CLICK| Chrome|Windows| 2017-07-23|       CLICK|
|U100364|WI100000401415|   2018-05-14| Pr100152|    null| Chrome|Windows| 2017-07-23|       CLICK|
|U100499|  WI1000004168|   2018-05-15| Pr101042|   CLICK| Chrome|Windows| 2017-02-08|       CLICK|
|U100499|  WI1000004168|   2018-05-19| Pr101042|   CLICK| Chrome|Windows| 2017-02-08|       CLICK|
|U100499|  WI1000004168|   2018-05-15|     null|   CLICK| Chrome|Windows| 2017-02-08|       CLICK|
|U100499|  WI1000004168|         null|     null|   CLICK| Chrome|Windows| 2017-02-08|       CLICK|
|U100499|  WI1000004168|   2018-05-23| pr101042|   CLICK| Chrome|Windows| 2017-02-08|       CLICK|
|U100499| 

In [154]:
# replacing null value in Activity column with the most frequent Activity based on UserID

join_df = join_df.withColumn('Activity',when(col('Activity').isNull(), col('Activity_New')).otherwise(col('Activity')))
join_df = join_df.drop('Activity_New')
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+
|U100364|WI100000401415|   2018-05-14| pr100152|   CLICK| Chrome|Windows| 2017-07-23|
|U100364|WI100000401415|   2018-05-14| Pr100152|   CLICK| Chrome|Windows| 2017-07-23|
|U100499|  WI1000004168|   2018-05-15| Pr101042|   CLICK| Chrome|Windows| 2017-02-08|
|U100499|  WI1000004168|   2018-05-19| Pr101042|   CLICK| Chrome|Windows| 2017-02-08|
|U100499|  WI1000004168|   2018-05-15|     null|   CLICK| Chrome|Windows| 2017-02-08|
|U100499|  WI1000004168|         null|     null|   CLICK| Chrome|Windows| 2017-02-08|
|U100499|  WI1000004168|   2018-05-23| pr101042|   CLICK| Chrome|Windows| 2017-02-08|
|U100499|  WI1000004168|   2018-05-21| Pr101042|   CLICK| Chrome|Windows| 2017-02-08|
|U100499|  WI1000004168|         null| pr101042|   CLI

In [155]:
join_df.select('OS').distinct().collect()

[Row(OS='android'),
 Row(OS='iOS'),
 Row(OS='ubuntu'),
 Row(OS='fedora'),
 Row(OS='Linux'),
 Row(OS='Fedora'),
 Row(OS='Chrome OS'),
 Row(OS='Ubuntu'),
 Row(OS='linux'),
 Row(OS='Tizen'),
 Row(OS='Android'),
 Row(OS='Mac OS X'),
 Row(OS='ios'),
 Row(OS='chrome os'),
 Row(OS='tizen'),
 Row(OS='Windows'),
 Row(OS='mac os x'),
 Row(OS='windows')]

In [156]:
# converting OS column values to uppercase

join_df = join_df.withColumn('OS', upper(col('OS')))
join_df.show()


+-------+--------------+-------------+---------+--------+-------+-------+-----------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+
|U100364|WI100000401415|   2018-05-14| pr100152|   CLICK| Chrome|WINDOWS| 2017-07-23|
|U100364|WI100000401415|   2018-05-14| Pr100152|   CLICK| Chrome|WINDOWS| 2017-07-23|
|U100499|  WI1000004168|   2018-05-15| Pr101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-19| Pr101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-15|     null|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|         null|     null|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-23| pr101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-21| Pr101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|         null| pr101042|   CLI

In [157]:
# converting ProductID column values to uppercase

join_df = join_df.withColumn('ProductID', upper(col('ProductID')))
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-19| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-15|     null|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|         null|     null|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-23| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-21| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|         null| PR101042|   CLI

In [158]:
# ProductID null value imputation

product_counts = join_df.groupBy(['UserID', 'ProductID']).agg(count(when(col('ProductID').isNotNull(), col('ProductID') ).otherwise(None)).alias('count'))

win = Window().partitionBy('UserID').orderBy(col('count').desc())
result = (product_counts
          .withColumn('row_num', row_number().over(win))
          .where(col('row_num') == 1)
          .select('UserID', 'ProductID')
         )
result = result.withColumnRenamed('ProductID','ProductID_New')
result.show()

+-------+-------------+
| UserID|ProductID_New|
+-------+-------------+
|U100364|     PR100152|
|U100499|     PR101042|
|U100608|     PR100283|
|U100611|     PR100152|
|U100754|     PR101062|
|U100999|     PR100017|
|U102315|     PR100027|
|U103221|     PR100483|
|U103279|     PR100017|
|U103647|     PR100390|
|U103859|     PR100252|
|U103874|     PR101042|
|U103924|     PR100945|
|U104238|     PR101042|
|U104440|     PR100055|
|U104625|     PR101744|
|U104644|     PR100771|
|U104722|     PR100166|
|U105187|     PR100220|
|U105380|     PR101800|
+-------+-------------+
only showing top 20 rows



In [159]:
join_df = join_df.join(result,['UserID'],'inner')
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+-------------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|ProductID_New|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+-------------+
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|     PR100152|
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|     PR100152|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|     PR101042|
|U100499|  WI1000004168|   2018-05-19| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|     PR101042|
|U100499|  WI1000004168|   2018-05-15|     null|   CLICK| Chrome|WINDOWS| 2017-02-08|     PR101042|
|U100499|  WI1000004168|         null|     null|   CLICK| Chrome|WINDOWS| 2017-02-08|     PR101042|
|U100499|  WI1000004168|   2018-05-23| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|     PR101042|


In [160]:
# replacing null value in Activity column with the most frequent Activity based on UserID

join_df = join_df.withColumn('ProductID',when(col('ProductID').isNull(), col('ProductID_New')).otherwise(col('ProductID')))
join_df = join_df.drop('ProductID_New')
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-19| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|         null| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-23| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|   2018-05-21| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|
|U100499|  WI1000004168|         null| PR101042|   CLI

In [161]:
join_df.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- webClientID: string (nullable = true)
 |-- VisitDateTime: date (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- Activity: string (nullable = true)
 |-- Browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- Signup Date: date (nullable = true)



In [162]:
# setting target date as 28-05-2018

import datetime as dt

target_date = dt.date(2018,5,28)
target_date


datetime.date(2018, 5, 28)

In [163]:
# computing the receny of visit of user w.r.t VisitDateTime 

join_df = join_df.withColumn('Duration', datediff(lit(target_date), 'VisitDateTime'))
join_df.show()


+-------+--------------+-------------+---------+--------+-------+-------+-----------+--------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|Duration|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+--------+
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499|  WI1000004168|   2018-05-19| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       9|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499|  WI1000004168|         null| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|    null|
|U100499|  WI1000004168|   2018-05-23| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       5|
|U100499|  WI1000004168|   2018-05-21| PR101042|  

In [164]:
join_df.select(mean('Duration').alias('mean')).show()

+------------------+
|              mean|
+------------------+
|11.081948550244755|
+------------------+



In [165]:
# Duration null value imputation

duration_counts = join_df.groupBy(['UserID', 'Duration']).agg(count(when(col('Duration').isNotNull(), col('Duration') ).otherwise(None)).alias('count'))

win = Window().partitionBy('UserID').orderBy(col('count').desc())
result = (duration_counts
          .withColumn('row_num', row_number().over(win))
          .where(col('row_num') == 1)
          .select('UserID', 'Duration')
         )
result = result.withColumnRenamed('Duration','Duration_New')
result.show()

+-------+------------+
| UserID|Duration_New|
+-------+------------+
|U100364|          14|
|U100499|           1|
|U100608|           7|
|U100611|          16|
|U100754|          12|
|U100999|           2|
|U102315|           6|
|U103221|           8|
|U103279|        null|
|U103647|           5|
|U103859|           2|
|U103874|           1|
|U103924|           7|
|U104238|           2|
|U104440|           1|
|U104625|           6|
|U104644|           1|
|U104722|          18|
|U105187|           1|
|U105380|           1|
+-------+------------+
only showing top 20 rows



In [166]:
join_df = join_df.join(result,['UserID'],'inner')
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+--------+------------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|Duration|Duration_New|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+--------+------------+
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|          14|
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|          14|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|           1|
|U100499|  WI1000004168|   2018-05-19| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       9|           1|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|           1|
|U100499|  WI1000004168|         null| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|    null|           1|
|U100499|  WI1000004168|   2

In [167]:
# replacing null value in Duration column with the most frequent Duration based on UserID

join_df = join_df.withColumn('Duration',when(col('Duration').isNull(), col('Duration_New')).otherwise(col('Duration')))
join_df = join_df.drop('Duration_New')
join_df.show()

+-------+--------------+-------------+---------+--------+-------+-------+-----------+--------+
| UserID|   webClientID|VisitDateTime|ProductID|Activity|Browser|     OS|Signup Date|Duration|
+-------+--------------+-------------+---------+--------+-------+-------+-----------+--------+
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100364|WI100000401415|   2018-05-14| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499|  WI1000004168|   2018-05-19| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       9|
|U100499|  WI1000004168|   2018-05-15| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499|  WI1000004168|         null| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499|  WI1000004168|   2018-05-23| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       5|
|U100499|  WI1000004168|   2018-05-21| PR101042|  

In [168]:
# dropping irrelevant columns 

join_df = join_df.drop('webClientID','VisitDateTime')
join_df.show()

+-------+---------+--------+-------+-------+-----------+--------+
| UserID|ProductID|Activity|Browser|     OS|Signup Date|Duration|
+-------+---------+--------+-------+-------+-----------+--------+
|U100364| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100364| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       9|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       5|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       7|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       6|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| 

In [169]:
# checking null values after initial imputation

join_df.select([count(when(isnull(c), c)).alias(c) for c in join_df.columns]).show()

+------+---------+--------+-------+---+-----------+--------+
|UserID|ProductID|Activity|Browser| OS|Signup Date|Duration|
+------+---------+--------+-------+---+-----------+--------+
|     0|      582|    1146|      0|  0|          0|     805|
+------+---------+--------+-------+---+-----------+--------+



In [170]:
join_df.filter((join_df.Duration.isNull()) & (join_df.Activity.isNull()) & (join_df.ProductID.isNull())).count()

7

In [171]:
join_df.filter((join_df.Activity.isNull()) ).show()

+-------+---------+--------+-------------+---------+-----------+--------+
| UserID|ProductID|Activity|      Browser|       OS|Signup Date|Duration|
+-------+---------+--------+-------------+---------+-----------+--------+
|U117600| PR102639|    null|       Chrome|  WINDOWS| 2018-04-13|      13|
|U121044| PR101471|    null|       Chrome|  WINDOWS| 2016-02-26|       9|
|U121044| PR101471|    null|       Chrome|  WINDOWS| 2016-02-26|       9|
|U129789| PR100017|    null|       Chrome|  WINDOWS| 2018-03-28|      16|
|U123054| PR100736|    null|       Chrome| MAC OS X| 2018-04-05|      10|
|U126052| PR100097|    null|Chrome Mobile|  ANDROID| 2016-06-13|      13|
|U128080| PR100374|    null|       Chrome|CHROME OS| 2018-03-03|      14|
|U128377| PR100739|    null|Chrome Mobile|  ANDROID| 2018-03-08|      14|
|U128516| PR101328|    null|Chrome Mobile|  ANDROID| 2017-04-10|    null|
|U136829| PR101702|    null|       Chrome|  WINDOWS| 2018-02-05|      21|
|U113477| PR100850|    null|Chrome Mob

In [172]:
join_df.filter((join_df.ProductID.isNull()) ).show()

+-------+---------+--------+-----------------+-------+-----------+--------+
| UserID|ProductID|Activity|          Browser|     OS|Signup Date|Duration|
+-------+---------+--------+-----------------+-------+-----------+--------+
|U134897|     null|PAGELOAD|           Chrome|WINDOWS| 2018-03-13|      20|
|U136638|     null|PAGELOAD|    Chrome Mobile|ANDROID| 2017-06-16|      21|
|U118337|     null|PAGELOAD|Chrome Mobile iOS|    IOS| 2018-03-18|       7|
|U128980|     null|   CLICK|           Chrome|WINDOWS| 2018-02-13|      15|
|U113880|     null|PAGELOAD|    Chrome Mobile|ANDROID| 2017-05-27|       5|
|U132712|     null|PAGELOAD|    Chrome Mobile|ANDROID| 2013-10-03|      18|
|U122750|     null|    null|    Chrome Mobile|ANDROID| 2017-08-26|    null|
|U102612|     null|PAGELOAD|           Chrome|WINDOWS| 2018-03-01|       9|
|U108677|     null|PAGELOAD|    Chrome Mobile|ANDROID| 2017-12-04|       3|
|U121936|     null|PAGELOAD|    Chrome Mobile|ANDROID| 2018-04-12|       9|
|U128451|   

In [173]:
join_df.groupBy('Activity').count().orderBy('count', ascending=False).show()


+--------+------+
|Activity| count|
+--------+------+
|   CLICK|467757|
|PAGELOAD|181792|
|    null|  1146|
+--------+------+



In [174]:
join_df.groupBy('ProductID').count().orderBy('count', ascending=False).show()


+---------+-----+
|ProductID|count|
+---------+-----+
| PR101042|89852|
| PR100166|26925|
| PR100102|25271|
| PR100017|22909|
| PR100390|21072|
| PR100200|12940|
| PR101367| 8952|
| PR100051| 8120|
| PR100123| 6308|
| PR100363| 4892|
| PR100072| 4626|
| PR100145| 4549|
| PR100287| 4104|
| PR100070| 3888|
| PR100086| 3847|
| PR100060| 3389|
| PR100036| 3288|
| PR100152| 3036|
| PR100219| 2940|
| PR100003| 2900|
+---------+-----+
only showing top 20 rows



In [175]:
join_df.groupBy('Duration').count().orderBy('count', ascending=False).show()


+--------+-----+
|Duration|count|
+--------+-----+
|      21|50038|
|      20|39071|
|       6|38189|
|       7|38168|
|       5|36468|
|       3|35940|
|       4|31334|
|      14|30347|
|      19|30288|
|      13|29362|
|       2|29324|
|      10|29132|
|       9|28648|
|      18|28264|
|      11|27640|
|       1|27049|
|      12|25429|
|       8|25189|
|      16|24690|
|      17|24370|
+--------+-----+
only showing top 20 rows



In [176]:
join_df.select(mean('Duration').alias('mean')).show()

+------------------+
|              mean|
+------------------+
|11.084862053578298|
+------------------+



In [177]:
# replacing the remaining null values with necessary values

join_df = join_df.fillna({'Activity':'CLICK', 'ProductID':'N/A'})
join_df = join_df.withColumn('Duration', when(col('Duration').isNull(),11).otherwise(col('Duration')))
join_df.show()

+-------+---------+--------+-------+-------+-----------+--------+
| UserID|ProductID|Activity|Browser|     OS|Signup Date|Duration|
+-------+---------+--------+-------+-------+-----------+--------+
|U100364| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100364| PR100152|   CLICK| Chrome|WINDOWS| 2017-07-23|      14|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       9|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|      13|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       5|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       7|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       6|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| PR101042|   CLICK| Chrome|WINDOWS| 2017-02-08|       1|
|U100499| 

In [178]:
join_df.select([count(when(isnull(c), c)).alias(c) for c in join_df.columns]).show()

+------+---------+--------+-------+---+-----------+--------+
|UserID|ProductID|Activity|Browser| OS|Signup Date|Duration|
+------+---------+--------+-------+---+-----------+--------+
|     0|        0|       0|      0|  0|          0|       0|
+------+---------+--------+-------+---+-----------+--------+



In [179]:
join_df.describe().show()

+-------+-------+---------+--------+--------------+-------+------------------+
|summary| UserID|ProductID|Activity|       Browser|     OS|          Duration|
+-------+-------+---------+--------+--------------+-------+------------------+
|  count| 650695|   650695|  650695|        650695| 650695|            650695|
|   mean|   null|     null|    null|          null|   null|11.084757067443272|
| stddev|   null|     null|    null|          null|   null| 6.308693590263574|
|    min|U100002|      N/A|   CLICK|   Amazon Silk|ANDROID|                 1|
|    max|U136965| PR112440|PAGELOAD|Yandex Browser|WINDOWS|                21|
+-------+-------+---------+--------+--------------+-------+------------------+



In [180]:
 # No_of_days_Visited_7_Days
    
visit_7_df = join_df.groupBy(['UserID']).agg(countDistinct(when(col('Duration') < 8, col('Duration') ).otherwise(None)).alias('COUNT'))
visit_7_df.show()

+-------+-----+
| UserID|COUNT|
+-------+-----+
|U100364|    0|
|U100499|    7|
|U100608|    3|
|U100611|    0|
|U100754|    2|
|U100999|    1|
|U102315|    6|
|U103221|    0|
|U103279|    0|
|U103647|    5|
|U103859|    3|
|U103874|    2|
|U103924|    2|
|U104238|    3|
|U104440|    1|
|U104625|    4|
|U104644|    2|
|U104722|    6|
|U105187|    1|
|U105380|    1|
+-------+-----+
only showing top 20 rows



In [181]:
No_of_days_Visited_7_Days_df = visit_7_df.withColumnRenamed('COUNT','No_of_days_Visited_7_Days')
No_of_days_Visited_7_Days_df.show()

+-------+-------------------------+
| UserID|No_of_days_Visited_7_Days|
+-------+-------------------------+
|U100364|                        0|
|U100499|                        7|
|U100608|                        3|
|U100611|                        0|
|U100754|                        2|
|U100999|                        1|
|U102315|                        6|
|U103221|                        0|
|U103279|                        0|
|U103647|                        5|
|U103859|                        3|
|U103874|                        2|
|U103924|                        2|
|U104238|                        3|
|U104440|                        1|
|U104625|                        4|
|U104644|                        2|
|U104722|                        6|
|U105187|                        1|
|U105380|                        1|
+-------+-------------------------+
only showing top 20 rows



In [182]:
# Pageloads_last_7_days

pageload_df = join_df.groupBy(['UserID']).agg(count(when((col('Duration') < 8) & (col('Activity')=='PAGELOAD'), col('Activity') ).otherwise(None)).alias('COUNT'))
pageload_df.show()

+-------+-----+
| UserID|COUNT|
+-------+-----+
|U100364|    0|
|U100499|  146|
|U100608|    4|
|U100611|    0|
|U100754|    8|
|U100999|    1|
|U102315|   11|
|U103221|    0|
|U103279|    0|
|U103647|   32|
|U103859|    4|
|U103874|   50|
|U103924|    3|
|U104238|   14|
|U104440|    0|
|U104625|    6|
|U104644|    3|
|U104722|   15|
|U105187|    3|
|U105380|    2|
+-------+-----+
only showing top 20 rows



In [183]:
Pageloads_last_7_days_df = pageload_df.withColumnRenamed('COUNT','Pageloads_last_7_days')
Pageloads_last_7_days_df.show()

+-------+---------------------+
| UserID|Pageloads_last_7_days|
+-------+---------------------+
|U100364|                    0|
|U100499|                  146|
|U100608|                    4|
|U100611|                    0|
|U100754|                    8|
|U100999|                    1|
|U102315|                   11|
|U103221|                    0|
|U103279|                    0|
|U103647|                   32|
|U103859|                    4|
|U103874|                   50|
|U103924|                    3|
|U104238|                   14|
|U104440|                    0|
|U104625|                    6|
|U104644|                    3|
|U104722|                   15|
|U105187|                    3|
|U105380|                    2|
+-------+---------------------+
only showing top 20 rows



In [184]:
# Clicks_last_7_days

click_df = join_df.groupBy(['UserID']).agg(count(when((col('Duration') < 8) & (col('Activity')=='CLICK'), col('Activity') ).otherwise(None)).alias('COUNT'))
click_df.show()

+-------+-----+
| UserID|COUNT|
+-------+-----+
|U100364|    0|
|U100499| 3117|
|U100608|    2|
|U100611|    0|
|U100754|   40|
|U100999|    2|
|U102315|   37|
|U103221|    0|
|U103279|    0|
|U103647|   16|
|U103859|   15|
|U103874|   24|
|U103924|    0|
|U104238|  197|
|U104440|    6|
|U104625|    3|
|U104644|   67|
|U104722|    0|
|U105187|    1|
|U105380|    2|
+-------+-----+
only showing top 20 rows



In [185]:
Clicks_last_7_days_df = click_df.withColumnRenamed('COUNT','Clicks_last_7_days')
Clicks_last_7_days_df.show()

+-------+------------------+
| UserID|Clicks_last_7_days|
+-------+------------------+
|U100364|                 0|
|U100499|              3117|
|U100608|                 2|
|U100611|                 0|
|U100754|                40|
|U100999|                 2|
|U102315|                37|
|U103221|                 0|
|U103279|                 0|
|U103647|                16|
|U103859|                15|
|U103874|                24|
|U103924|                 0|
|U104238|               197|
|U104440|                 6|
|U104625|                 3|
|U104644|                67|
|U104722|                 0|
|U105187|                 1|
|U105380|                 2|
+-------+------------------+
only showing top 20 rows



In [186]:
# Most_Active_OS

os_df = join_df.groupBy(['UserID','OS']).agg(count(col('OS')).alias('most_active'))
w = Window().partitionBy('UserID').orderBy(desc('most_active'))
df_with_rank = (os_df.withColumn('row_number', row_number().over(w)))
result_df = df_with_rank.where((col('row_number') == 1))
result_df.show()

+-------+-------+-----------+----------+
| UserID|     OS|most_active|row_number|
+-------+-------+-----------+----------+
|U100364|WINDOWS|          2|         1|
|U100499|WINDOWS|       4615|         1|
|U100608|ANDROID|          8|         1|
|U100611|ANDROID|          4|         1|
|U100754|WINDOWS|         90|         1|
|U100999|ANDROID|          4|         1|
|U102315|WINDOWS|         75|         1|
|U103221|WINDOWS|          3|         1|
|U103279|ANDROID|          1|         1|
|U103647|WINDOWS|         92|         1|
|U103859|WINDOWS|         21|         1|
|U103874|ANDROID|         74|         1|
|U103924|ANDROID|          3|         1|
|U104238|  LINUX|        218|         1|
|U104440|WINDOWS|          6|         1|
|U104625|WINDOWS|         15|         1|
|U104644|WINDOWS|         72|         1|
|U104722|WINDOWS|         56|         1|
|U105187|WINDOWS|          4|         1|
|U105380|WINDOWS|          4|         1|
+-------+-------+-----------+----------+
only showing top

In [187]:
Most_Active_OS_df = result_df['UserID','OS'].withColumnRenamed('OS','Most_Active_OS')
Most_Active_OS_df.show()

+-------+--------------+
| UserID|Most_Active_OS|
+-------+--------------+
|U100364|       WINDOWS|
|U100499|       WINDOWS|
|U100608|       ANDROID|
|U100611|       ANDROID|
|U100754|       WINDOWS|
|U100999|       ANDROID|
|U102315|       WINDOWS|
|U103221|       WINDOWS|
|U103279|       ANDROID|
|U103647|       WINDOWS|
|U103859|       WINDOWS|
|U103874|       ANDROID|
|U103924|       ANDROID|
|U104238|         LINUX|
|U104440|       WINDOWS|
|U104625|       WINDOWS|
|U104644|       WINDOWS|
|U104722|       WINDOWS|
|U105187|       WINDOWS|
|U105380|       WINDOWS|
+-------+--------------+
only showing top 20 rows



In [188]:
# User_Vintage

vintage_df = join_df.groupBy(['UserID','Signup Date']).count()
vintage_df = vintage_df.withColumn('Vintage', datediff(lit(target_date), 'Signup Date'))
vintage_df.show()


+-------+-----------+-----+-------+
| UserID|Signup Date|count|Vintage|
+-------+-----------+-----+-------+
|U100364| 2017-07-23|    2|    309|
|U100499| 2017-02-08| 4620|    474|
|U100608| 2017-03-01|    8|    453|
|U100611| 2016-06-30|    8|    697|
|U100754| 2016-08-24|   90|    642|
|U100999| 2013-04-03|    4|   1881|
|U102315| 2018-03-03|   75|     86|
|U103221| 2018-04-03|    3|     55|
|U103279| 2018-01-27|    1|    121|
|U103647| 2018-02-17|   92|    100|
|U103859| 2015-05-29|   21|   1095|
|U103874| 2015-12-31|   75|    879|
|U103924| 2017-10-06|    3|    234|
|U104238| 2017-03-30|  218|    424|
|U104440| 2018-02-01|    6|    116|
|U104625| 2016-08-10|   15|    656|
|U104644| 2018-03-29|   72|     60|
|U104722| 2018-03-24|   56|     65|
|U105187| 2018-02-18|    4|     99|
|U105380| 2015-11-21|    4|    919|
+-------+-----------+-----+-------+
only showing top 20 rows



In [189]:
User_Vintage_df = vintage_df['UserID','Vintage'].withColumnRenamed('Vintage','User_Vintage')
User_Vintage_df.show()

+-------+------------+
| UserID|User_Vintage|
+-------+------------+
|U100364|         309|
|U100499|         474|
|U100608|         453|
|U100611|         697|
|U100754|         642|
|U100999|        1881|
|U102315|          86|
|U103221|          55|
|U103279|         121|
|U103647|         100|
|U103859|        1095|
|U103874|         879|
|U103924|         234|
|U104238|         424|
|U104440|         116|
|U104625|         656|
|U104644|          60|
|U104722|          65|
|U105187|          99|
|U105380|         919|
+-------+------------+
only showing top 20 rows



In [190]:
# No_Of_Products_Viewed_15_Days

visit_15_df = join_df.groupBy(['UserID']).agg(countDistinct(when((col('Duration') < 16) & (col('Activity')=='PAGELOAD') & (col('ProductID')!='N/A'), col('ProductID') ).otherwise(None)).alias('COUNT'))
visit_15_df.show()


+-------+-----+
| UserID|COUNT|
+-------+-----+
|U100364|    0|
|U100499|   42|
|U100608|    3|
|U100611|    1|
|U100754|    8|
|U100999|    1|
|U102315|    3|
|U103221|    1|
|U103279|    1|
|U103647|    3|
|U103859|    4|
|U103874|    6|
|U103924|    2|
|U104238|    7|
|U104440|    0|
|U104625|    7|
|U104644|    2|
|U104722|    1|
|U105187|    2|
|U105380|    2|
+-------+-----+
only showing top 20 rows



In [191]:
No_Of_Products_Viewed_15_Days_df = visit_15_df.withColumnRenamed('COUNT','No_Of_Products_Viewed_15_Days')
No_Of_Products_Viewed_15_Days_df.show()

+-------+-----------------------------+
| UserID|No_Of_Products_Viewed_15_Days|
+-------+-----------------------------+
|U100364|                            0|
|U100499|                           42|
|U100608|                            3|
|U100611|                            1|
|U100754|                            8|
|U100999|                            1|
|U102315|                            3|
|U103221|                            1|
|U103279|                            1|
|U103647|                            3|
|U103859|                            4|
|U103874|                            6|
|U103924|                            2|
|U104238|                            7|
|U104440|                            0|
|U104625|                            7|
|U104644|                            2|
|U104722|                            1|
|U105187|                            2|
|U105380|                            2|
+-------+-----------------------------+
only showing top 20 rows



In [192]:
# Most_Viewed_product_15_Days

most_view_prod_df = join_df.groupBy(['UserID','ProductID','Duration']).agg(count(when((col('Duration') < 16) & (col('Activity')=='PAGELOAD') & (col('ProductID')!='N/A'), col('ProductID') ).otherwise(None)).alias('most_frequent')).orderBy('most_frequent', ascending=False)
w = Window().partitionBy('UserID').orderBy(asc('Duration'))
df_with_rank = (most_view_prod_df.withColumn('row_number', row_number().over(w)))
result_df = df_with_rank.where((col('row_number') == 1))
result_df = result_df.withColumn('ProductID', when((col('most_frequent')==0),'Product101').otherwise(col('ProductID')))
result_df.show()

+-------+----------+--------+-------------+----------+
| UserID| ProductID|Duration|most_frequent|row_number|
+-------+----------+--------+-------------+----------+
|U100364|Product101|      14|            0|         1|
|U100499|  PR101042|       1|           17|         1|
|U100608|  PR100341|       1|            1|         1|
|U100611|  PR100138|      10|            1|         1|
|U100754|  PR100166|       3|            1|         1|
|U100999|  PR100017|       2|            1|         1|
|U102315|  PR100027|       1|            1|         1|
|U103221|  PR100483|       8|            2|         1|
|U103279|  PR100017|      11|            1|         1|
|U103647|  PR100390|       1|            1|         1|
|U103859|  PR100252|       2|            1|         1|
|U103874|  PR101042|       1|           13|         1|
|U103924|  PR101039|       4|            1|         1|
|U104238|  PR101367|       1|            1|         1|
|U104440|Product101|       1|            0|         1|
|U104625| 

In [193]:
Most_Viewed_product_15_Days_df = result_df['UserID','ProductID'].withColumnRenamed('ProductID','Most_Viewed_product_15_Days')
Most_Viewed_product_15_Days_df.show()

+-------+---------------------------+
| UserID|Most_Viewed_product_15_Days|
+-------+---------------------------+
|U100364|                 Product101|
|U100499|                   PR101042|
|U100608|                   PR100341|
|U100611|                   PR100138|
|U100754|                   PR100166|
|U100999|                   PR100017|
|U102315|                   PR100027|
|U103221|                   PR100483|
|U103279|                   PR100017|
|U103647|                   PR100390|
|U103859|                   PR100252|
|U103874|                   PR101042|
|U103924|                   PR101039|
|U104238|                   PR101367|
|U104440|                 Product101|
|U104625|                   PR100210|
|U104644|                   PR100771|
|U104722|                   PR100166|
|U105187|                   PR100017|
|U105380|                   PR101800|
+-------+---------------------------+
only showing top 20 rows



In [194]:
# Recently_Viewed_Product

visit_recent_df = join_df.groupBy(['UserID','ProductID','Duration']).agg(count(when((col('Activity')=='PAGELOAD') & (col('ProductID')!='N/A') , col('Duration') ).otherwise(None)).alias('count'))
w = Window().partitionBy('UserID').orderBy(asc('Duration'))
df_with_rank = (visit_recent_df.withColumn('row_number', row_number().over(w)))
result_df = df_with_rank.where((col('row_number') == 1))
result_df = result_df.withColumn('ProductID', when((col('count')==0) ,'Product101').otherwise(col('ProductID')))
result_df.show()

+-------+----------+--------+-----+----------+
| UserID| ProductID|Duration|count|row_number|
+-------+----------+--------+-----+----------+
|U100364|Product101|      14|    0|         1|
|U100499|  PR101042|       1|   17|         1|
|U100608|  PR100341|       1|    1|         1|
|U100611|  PR100138|      10|    1|         1|
|U100754|  PR100166|       3|    1|         1|
|U100999|  PR100017|       2|    1|         1|
|U102315|  PR100027|       1|    1|         1|
|U103221|  PR100483|       8|    2|         1|
|U103279|  PR100017|      11|    1|         1|
|U103647|  PR100390|       1|    1|         1|
|U103859|Product101|       2|    0|         1|
|U103874|  PR101042|       1|   13|         1|
|U103924|  PR101039|       4|    1|         1|
|U104238|Product101|       1|    0|         1|
|U104440|Product101|       1|    0|         1|
|U104625|  PR100210|       1|    1|         1|
|U104644|  PR100771|       1|    1|         1|
|U104722|  PR100166|       1|    2|         1|
|U105187|  PR

In [195]:
Recently_Viewed_Product_df = result_df['UserID','ProductID'].withColumnRenamed('ProductID','Recently_Viewed_Product')
Recently_Viewed_Product_df.show()

+-------+-----------------------+
| UserID|Recently_Viewed_Product|
+-------+-----------------------+
|U100364|             Product101|
|U100499|               PR101042|
|U100608|               PR100341|
|U100611|               PR100138|
|U100754|               PR100166|
|U100999|               PR100017|
|U102315|               PR100027|
|U103221|               PR100483|
|U103279|               PR100017|
|U103647|               PR100390|
|U103859|             Product101|
|U103874|               PR101042|
|U103924|               PR101039|
|U104238|             Product101|
|U104440|             Product101|
|U104625|               PR100210|
|U104644|               PR100771|
|U104722|               PR100166|
|U105187|               PR100220|
|U105380|               PR101800|
+-------+-----------------------+
only showing top 20 rows



In [196]:
# combining all the dataframes to get the final dataset

target_df = No_of_days_Visited_7_Days_df.join(No_Of_Products_Viewed_15_Days_df,['UserID'],'inner').join(User_Vintage_df,['UserID'],'inner') \
            .join(Most_Viewed_product_15_Days_df,['UserID'],'inner').join(Most_Active_OS_df,['UserID'],'inner').join(Recently_Viewed_Product_df,['UserID'],'inner') \
            .join(Pageloads_last_7_days_df,['UserID'],'inner').join(Clicks_last_7_days_df,['UserID'],'inner')


In [197]:
target_df.show()

+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
| UserID|No_of_days_Visited_7_Days|No_Of_Products_Viewed_15_Days|User_Vintage|Most_Viewed_product_15_Days|Most_Active_OS|Recently_Viewed_Product|Pageloads_last_7_days|Clicks_last_7_days|
+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|U100364|                        0|                            0|         309|                 Product101|       WINDOWS|             Product101|                    0|                 0|
|U100499|                        7|                           42|         474|                   PR101042|       WINDOWS|               PR101042|                  146|              3117|
|U100608|                        3|                            3|

In [198]:
target_df.count()

34050

In [199]:
# checking for null values

target_df.select([count(when(isnull(c), c)).alias(c) for c in target_df.columns]).show()

+------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|UserID|No_of_days_Visited_7_Days|No_Of_Products_Viewed_15_Days|User_Vintage|Most_Viewed_product_15_Days|Most_Active_OS|Recently_Viewed_Product|Pageloads_last_7_days|Clicks_last_7_days|
+------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|     0|                        0|                            0|           0|                          0|             0|                      0|                    0|                 0|
+------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+



In [200]:
# sorting based on UserID

target_df = target_df.sort(target_df.UserID.asc())
target_df.show()

+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
| UserID|No_of_days_Visited_7_Days|No_Of_Products_Viewed_15_Days|User_Vintage|Most_Viewed_product_15_Days|Most_Active_OS|Recently_Viewed_Product|Pageloads_last_7_days|Clicks_last_7_days|
+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|U100002|                        0|                            2|          53|                   PR100258|       ANDROID|               PR100258|                    0|                 0|
|U100003|                        1|                            1|        1021|                   PR100079|       WINDOWS|               PR100079|                    1|                 2|
|U100004|                        1|                           13|

In [202]:
# export as csv for submission

target_df.toPandas().to_csv("sub7.csv", index=False)
